In [ ]:
#Allow to install random packages
import sys
!{sys.executable} -m pip install pandas-datareader

In [ ]:
#Import our data
import pathlib
import pandas as pd
cwd = pathlib.Path.cwd()

code_directory = cwd.parents[1]

bas_directory = code_directory / "notebooks" / "Bas"
data_file = bas_directory / "df_filtered_maize_trade_oil_weather_futures.xlsx"
data_file

df = pd.read_excel(data_file, header=[0, 1], index_col=0)
df.head(5)

In [ ]:
#Split dataset into test 20% and train 80%
n = len(df)
df_train = df[0:int(n*0.8)]
#val_df = df[int(n*0.7):int(n*0.9)]
df_test = df[int(n*0.8):]
print(df_train.head(5))

In [ ]:
df_train.dtypes

In [121]:
#Apply Granger causality to test if lagged variables
from statsmodels.tsa.stattools import grangercausalitytests

maxlag = 2
variables= df.columns[0:(len(df.columns)-1)] #Ensure timecolumn is removed from data
matrix = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
for col in matrix.columns:
    for row in matrix.index:
        test_result = grangercausalitytests(df[[row, col]], maxlag=20, verbose=False)
        p_values = [round(test_result[i+1][0]['ssr_chi2test'][1],4) for i in range(maxlag)]
        min_p_value = np.min(p_values)
        matrix.loc[row, col] = min_p_value
matrix.columns = [var + '_x' for var in variables]
matrix.index = [var + '_y' for var in variables]
print(matrix)

         rgnp_x  pgnp_x   ulc_x  gdfco_x   gdf_x  gdfim_x  gdfcf_x  gdfce_x
rgnp_y   1.0000  0.1849  0.0081   0.0360  0.0093   0.2181   0.6115   0.2297
pgnp_y   0.0000  1.0000  0.0000   0.0000  0.0000   0.0000   0.0000   0.0000
ulc_y    0.0000  0.0000  1.0000   0.0002  0.0000   0.0000   0.0000   0.0949
gdfco_y  0.0000  0.0000  0.0000   1.0000  0.0000   0.0000   0.0000   0.0000
gdf_y    0.0000  0.0000  0.0000   0.0000  1.0000   0.0000   0.0000   0.0090
gdfim_y  0.0011  0.0067  0.0014   0.0083  0.0019   1.0000   0.0150   0.0000
gdfcf_y  0.0000  0.0000  0.0008   0.0008  0.0000   0.0057   1.0000   0.0030
gdfce_y  0.0673  0.0485  0.0017   0.0002  0.0000   0.0000   0.0042   1.0000


In [ ]:
#Stationary testing by using Augmented Dickey-Fuller Test we might need to do decomposition ?? DUH weather is trendy?
from statsmodels.tsa.stattools import adfuller

def adfuller_test(series, sig=0.05, name=''):
    res = adfuller(series, autolag='AIC')
    p_value = round(res[1], 3)

    if p_value <= sig:
        print(f" {name} : P-Value = {p_value} => Stationary. ")
    else:
        print(f" {name} : P-Value = {p_value} => Non-stationary.")

for name, column in df.iteritems():
    adfuller_test(column, name=column.name)

In [128]:
#Import test data
import pandas as pd
import numpy as np
import statsmodels.api as sm

df = pd.read_excel('Raotbl6.xlsx', index_col='date', parse_dates=True)
print(df.head(10))

              rgnp    pgnp   ulc  gdfco   gdf  gdfim  gdfcf  gdfce
date                                                              
1956-04-01  1606.4  1608.3  47.5   36.9  37.4   26.9   32.3   23.1
1956-07-01  1637.0  1622.2  47.5   37.4  37.5   27.0   32.2   23.4
1956-10-01  1629.5  1636.2  48.7   37.6  37.6   27.1   32.4   23.4
1957-01-01  1643.4  1650.3  48.8   37.7  37.8   27.1   32.5   23.8
1957-04-01  1671.6  1664.6  49.1   37.8  37.8   27.2   32.4   23.8
1957-07-01  1666.8  1679.0  49.6   38.0  38.0   27.4   32.8   23.9
1957-10-01  1668.4  1693.5  50.0   38.1  38.1   27.4   32.9   24.1
1958-01-01  1654.1  1708.2  50.2   38.2  38.2   27.2   33.2   24.2
1958-04-01  1671.3  1722.9  50.1   38.2  38.2   27.2   33.2   24.2
1958-07-01  1692.1  1737.8  49.8   38.3  38.2   27.2   33.2   24.2


In [130]:
# Define the seasonal cycle length
s = 12 # for monthly data

# Define the model order and seasonal order
p = 2 # AR order
q = 2 # MA order
P = 1 # seasonal AR order
Q = 1 # seasonal MA order

In [133]:
# Create the multivariate seasonal ARMA model

Y = df[['rgnp', 'pgnp']]
X = df.loc[:, df.columns != 'rgnp']
X = X.loc[:, X.columns != 'pgnp']
print(Y.head(5))
print(X.head(5))

              rgnp    pgnp
date                      
1956-04-01  1606.4  1608.3
1956-07-01  1637.0  1622.2
1956-10-01  1629.5  1636.2
1957-01-01  1643.4  1650.3
1957-04-01  1671.6  1664.6
             ulc  gdfco   gdf  gdfim  gdfcf  gdfce
date                                              
1956-04-01  47.5   36.9  37.4   26.9   32.3   23.1
1956-07-01  47.5   37.4  37.5   27.0   32.2   23.4
1956-10-01  48.7   37.6  37.6   27.1   32.4   23.4
1957-01-01  48.8   37.7  37.8   27.1   32.5   23.8
1957-04-01  49.1   37.8  37.8   27.2   32.4   23.8


In [138]:
from statsmodels.tsa.statespace.varmax import VARMAX

#model = sm.tsa.statespace.SARIMAX(endog, order=(p,0,q), seasonal_order=(P,0,Q,s), trend='c')
model = VARMAX(endog = df, order=(p,q), trend='c')


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/statespace/varmax.py:161: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  warnings.warn('No frequency information was'


In [137]:

# Fit the model to the data
results = model.fit()

# Print the model summary
print(results.summary())

# Make a forecast for the next 12 months
forecast = results.forecast(steps=12)

# Plot the forecast
data.plot()
forecast.plot()
plt.show()

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                           Statespace Model Results                           
Dep. Variable:       ['rgnp', 'pgnp']   No. Observations:                  123
Model:                    VARMAX(2,2)   Log Likelihood               -1157.021
                          + intercept   AIC                           2380.043
Date:                Fri, 17 Feb 2023   BIC                           2472.845
Time:                        12:26:13   HQIC                          2417.739
Sample:                    04-01-1956                                         
                         - 10-01-1986                                         
Covariance Type:                  opg                                         
Ljung-Box (L1) (Q):             3.48, 1.26   Jarque-Bera (JB):           2.55, 8.52
Prob(Q):                        0.06, 0.26   Prob(JB):                   0.28, 0.01
Heteroskedasticity (H):         1.59, 2.62   Skew:                      -0.18, 0.09
Prob(H) (two-sided):            0.14,

ValueError: Out-of-sample operations in a model with a regression component require additional exogenous values via the `exog` argument.